In [1]:
import findspark
findspark.init("C:\Spark")
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
sc = SparkContext("local", "Simple")

7-15 PAT JUDGE

https://pintia.cn/problem-sets/16/problems/677

In [2]:
patSample = sc.textFile("patSample.txt")

In [3]:
index_sample = patSample.zipWithIndex()

In [4]:
data = index_sample.filter(lambda x: x[1] > 1).map(lambda x: x[0].split(" "))

In [5]:
info = index_sample.filter(lambda x: x[1] == 0).map(lambda x: x[0].split(" "))

In [6]:
fullMark = index_sample.filter(lambda x: x[1] == 1).map(lambda x: x[0].split(" "))

In [7]:
fullMarkB = sc.broadcast(fullMark.collect())

In [8]:
userId_high = data.map(lambda x: ((x[0], x[1]), x[2])).reduceByKey(lambda x, y: x> y and x or y)

In [9]:
print(userId_high.collect())

[(('00002', '2'), '25'), (('00007', '4'), '17'), (('00005', '1'), '20'), (('00007', '2'), '25'), (('00001', '1'), '18'), (('00004', '3'), '25'), (('00005', '3'), '22'), (('00006', '4'), '-1'), (('00001', '2'), '18'), (('00002', '1'), '20'), (('00004', '1'), '15'), (('00002', '4'), '18'), (('00001', '3'), '4'), (('00001', '4'), '2'), (('00005', '2'), '-1'), (('00004', '2'), '0')]


In [10]:
user_sum = userId_high.map(lambda x:(x[0][0], x[1])).reduceByKey(lambda x, y: int(x)+int(y) if y!="-1" else int(x)+0)

In [11]:
print(user_sum.collect())

[('00002', 63), ('00007', 42), ('00005', 42), ('00001', 42), ('00004', 40), ('00006', '-1')]


In [12]:
userID_high_group = userId_high.map(lambda x: (x[0][0], x[1])).groupByKey()

In [13]:
userID_sum_high = user_sum.join(userID_high_group).map(lambda x:(x[0],(x[1][0],list(x[1][1]))))

In [14]:
print(userID_sum_high.collect())

[('00002', (63, ['25', '20', '18'])), ('00007', (42, ['17', '25'])), ('00005', (42, ['20', '22', '-1'])), ('00001', (42, ['18', '18', '4', '2'])), ('00004', (40, ['25', '15', '0'])), ('00006', ('-1', ['-1']))]


In [15]:
print(userId_high.map(lambda x: (x[0][0], (x[0][1], x[1]))).collect())

[('00002', ('2', '25')), ('00007', ('4', '17')), ('00005', ('1', '20')), ('00007', ('2', '25')), ('00001', ('1', '18')), ('00004', ('3', '25')), ('00005', ('3', '22')), ('00006', ('4', '-1')), ('00001', ('2', '18')), ('00002', ('1', '20')), ('00004', ('1', '15')), ('00002', ('4', '18')), ('00001', ('3', '4')), ('00001', ('4', '2')), ('00005', ('2', '-1')), ('00004', ('2', '0'))]


In [16]:
joined_score = userID_sum_high.map(lambda x: (x[0], x[1])).join(userId_high.map(lambda x: (x[0][0], (x[0][1], x[1]))))

In [17]:
def rank(x):
    question_id = x[1][1][0].split(",")
    question_score = x[1][1][1].split(",")
    perfect = 0
    fm = fullMarkB.value
    for index, item in enumerate(question_id):
        if int(question_score[index]) != int(fm[0][int(item)-1]):
            continue
        perfect += 1
    return [x[0], str(perfect)]

In [18]:
score_rdd = joined_score.map(lambda x: rank(x)).reduceByKey(lambda x, y: int(x) + int(y) )

In [19]:
print(score_rdd.collect())

[('00005', 1), ('00006', '0'), ('00002', 2), ('00007', 1), ('00001', 0), ('00004', 1)]


In [20]:
data_final = userID_sum_high.join(score_rdd).filter(lambda x: x[1][0][0] != '-1').sortBy(lambda x: x[1][1], ascending=False).sortBy(lambda x: x[1][0][0], ascending=False)

In [21]:
print(data_final.collect())

[('00002', ((63, ['25', '20', '18']), 2)), ('00005', ((42, ['20', '22', '-1']), 1)), ('00007', ((42, ['17', '25']), 1)), ('00001', ((42, ['18', '18', '4', '2']), 0)), ('00004', ((40, ['25', '15', '0']), 1))]


In [30]:
data_ranked = data_final.map(lambda x: (x[0], x[1][0][0], x[1][0][1])).collect()

In [31]:
for a, b, c in data_ranked:
    print(a, b, c)

00002 63 ['25', '20', '18']
00005 42 ['20', '22', '-1']
00007 42 ['17', '25']
00001 42 ['18', '18', '4', '2']
00004 40 ['25', '15', '0']
